In [1]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

Enter API key for Groq:  ········


In [ ]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.chat_models import init_chat_model
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import os

# Path to the CSV file
output_file_path = "/data/output.csv"

# Load the CSV file
df = pd.read_csv(output_file_path)

# Ensure the CSV file has a column named 'text'
if 'text' not in df.columns:
    raise ValueError("CSV file must have a 'text' column")

# Extract sentences from the CSV file
sentences = df['text'].tolist()

# Load a pre-trained sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose any model you prefer

# Generate embeddings for the sentences
embeddings = model.encode(sentences).astype('float32')

# Create a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(embeddings)  # Add embeddings to the index

def retrieve_similar_sentences(query_sentence, k=1):
    # Generate embedding for the query sentence
    query_embedding = model.encode(query_sentence).astype('float32').reshape(1, -1)  # Reshape to 2D array

    # Search the index
    distances, indices = index.search(query_embedding, k)

    # Retrieve and return the most similar sentences
    similar_sentences = [sentences[indices[0][i]] for i in range(k)]
    return similar_sentences

def main():
    # Set OpenAI API key (ensure you have set the environment variable OPENAI_API_KEY)
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        print("Error: Please set your OPENAI_API_KEY environment variable.")
        return

    # Initialize LLM
    chat = init_chat_model("llama3-8b-8192", model_provider="groq")
    
    # Initial system message
    messages = [SystemMessage(content="You are a helpful AI assistant.")]
    
    print("Chatbot is ready! Type 'exit' to quit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        
        # Retrieve similar sentences based on user input
        similar_sentences = retrieve_similar_sentences(user_input)
        context = " ".join(similar_sentences)  # Combine similar sentences for context
        
        messages.append(HumanMessage(content=user_input))
        messages.append(SystemMessage(content=f"Context: {context}"))  # Add context to the messages
        
        # Generate response
        response = chat.invoke(messages)
        ai_message = AIMessage(content=response.content)
        messages.append(ai_message)
        
        print(f"AI: {response.content}")

if __name__ == "__main__":
    main()
